In [15]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [17]:
import folium 
import wget
import pandas as pd 

In [18]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [19]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file) 

In [20]:
spacex_df = spacex_df[["Launch Site", "Lat", "Long", "class"]]
launch_sites_df = spacex_df.groupby(["Launch Site"], as_index=False).first()
launch_sites_df = launch_sites_df[["Launch Site", "Lat", "Long"]]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [21]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)


In [22]:
circle = folium.Circle(nasa_coordinate, radius=2000, color= '#d35400', fill=True).add_child(folium.Popup("NASA Johnson Space Center"))

marker = folium.map.Marker(
    nasa_coordinate, 

    icon= DivIcon( 
        icon_size=(20,20),
        icon_anchor=(0,0), 
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )

    )

site_map.add_child(circle)
site_map.add_child(marker)



In [25]:
# Reinitialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle and marker for each launch site in the launch_sites_df dataframe
for index, row in launch_sites_df.iterrows():
    # Extract latitude and longitude
    site_coordinate = [row['Lat'], row['Long']]
    
    # Create a circle object for the launch site
    circle = folium.Circle(site_coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker object for the launch site
    marker = folium.map.Marker(
        site_coordinate,
        icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
                     html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'])
    )
    
    # Add the circle and marker objects to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map


In [26]:
spacex_df.tail()

,Launch Site,Lat,Long,class
51,CCAFS SLC-40,28.563197,-80.57682,0
52,CCAFS SLC-40,28.563197,-80.57682,0
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


In [27]:
marker_cluster = MarkerCluster()

In [29]:
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")
spacex_df.head() 

"""
Yukarıdaki Kodun Daha Geniş Hali 

def assign_marker_color(launch_outcome): 
    if launch_outcome == 1:
        return "green"
    else: 
        return "red"
    
spacex_df["marker_color"] = spacex_df["class"].apply(assign_marker_color)
spacex_df.tail()

"""

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [34]:
marker_cluster = MarkerCluster()

site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows(): 
    marker = folium.Marker(
        location=[record["Lat"], record["Long"]], 
        icon=folium.Icon(color="white", icon_color=record["marker_color"])
    )
    marker_cluster.add_child(marker)
site_map